# urllib库基础使用

* urllib有4个模块
    * request是请求HTTP的
    * error是进行异常处理
    * parse处理url
    * robotparser是处理爬虫是否可爬的

In [ ]:
import urllib.request
import urllib.error
import urllib.robotparser
import urllib.parse
import socket

get方式请求返回HTTPResposne类型对象

respons的方法有：

1. read（）读取，byet类型
2. getheader(name)获取响应头中的name属性值
3. status响应状态码

In [ ]:
response=urllib.request.urlopen("http://www.baidu.com")
print(response.read().decode("utf-8"))

post方式请求：
* 传递的数据要是字节流
* 制定编码格式
* 如果要使用字典转换字节流需要使用urllib.parse.urlencode()
* 除了data还有context传递ssl设置     
* cafile和capath设定ca证书和路经用于https

In [ ]:
a={"a":"b"}
urllib.parse.urlencode(a)

In [ ]:
data=bytes("a=b",encoding="utf-8")
response=urllib.request.urlopen("http://httpbin.org/post",data=data)
print(response.read().decode("utf-8"))

设置请求超时时间，超时后会报错，并判断错误类型是是否是超时类型

In [ ]:
try:
    response=urllib.request.urlopen("http://www.baidu.com",timeout=0.01)
except urllib.error.URLError as e:
    print(e)
    if isinstance(e.reason,socket.timeout):

        print("timeout")

# urllib库高级使用-添加请求头、设置cookie、代理、基本账户验证

使用Request类将请求的uel（网站），headers（请求头），data（post传输的数据），method请求方式组合成一个类

headers还可以调用实例的.add_headers来添加

In [ ]:
url="http://httpbin.org/post"
headers={"User-Agent":"Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.132 Safari/537.36"}
req=urllib.request.Request(url,headers=headers,data=data,method="POST")
response=urllib.request.urlopen(req)
print(response.read().decode("utf-8"))

In [ ]:
import http.cookiejar

验证——用于对付弹出身份验证
    
实例化密码管理对象，并通过add_password方法添加对应的账户密码与url，再继承到认证管理类的handler，

在用buli_opener将handler转换成opener，最后使用open方法请求

！！！弹出身份校验没有实际使用过！！！    

In [ ]:

uername="username"
password="pasword"
url="http://localhost:5000/"
p=urllib.request.HTTPPasswordMgrWithDefaultRealm()
p.add_password(None,url,uername,password)
auth_handler=urllib.request.HTTPBasicAuthHandler(p)
opener=urllib.request.build_opener(auth_handler)
try:
    result=opener.open(url)
    html=result.read().decode("utf-8")
    print(html)
except urllib.error.URLError as e :
    print(e.reason)

代理

定义代理实例并通过build_opener转换再使用open请求

在https://ip.ihuan.me/ 中找到对应的代理服务器

In [ ]:
proxy=urllib.request.ProxyHandler({"http":"113.195.140.57:8085","https":"127.0.0.1:80"})
opener=urllib.request.build_opener(proxy)
try:
    reponse=opener.open("http://httpbin.org/get")
    html=reponse.read().decode("utf-8")
    print(html)
except urllib.error.URLError as e :
    print(e.reason)

获取cookies

生成cookies罐集成给cookie处理类，再转换成openre，最后使用open方法请求，可以使最开始的cookies罐实例有cookies

In [ ]:
cookie=http.cookiejar.CookieJar()
handler=urllib.request.HTTPCookieProcessor(cookie)
opener=urllib.request.build_opener(handler)
response=opener.open("http://www.baidu.com")
print(cookie)

In [ ]:
获取浏览器格式cookies并保存到c.txt文件中

与获取先同结构只是修改cookies罐类型为浏览器cookie罐并调用save方法保存，其中ignore忽略丢弃与到期
cookies文件还有lwp格式

cookie=http.cookiejar.MozillaCookieJar(fil)替换成cookie=http.cookiejar.LWPCookieJar

In [ ]:
fil="cookies.txt"
cookie=http.cookiejar.MozillaCookieJar(fil)
handler=urllib.request.HTTPCookieProcessor(cookie)
opener=urllib.request.build_opener(handler)
response=opener.open("http://ww.baidu.com")
cookie.save(ignore_discard=True,ignore_expires=True)

也可以使用load方法调用读取文件中的cookie文件

In [ ]:
ookie=http.cookiejar.MozillaCookieJar()
cookie.load("cookies.txt",ignore_discard=True,ignore_expires=True)
print(cookie)

# urllib错误

HTTPError是URLError的子类来处理http请求出现的错误

1. reason属性是返回错误原因
2. code属性是返回请求状态码
3. headers属性返回请求头的信息

In [ ]:
try:
    response=urllib.request.urlopen("http://www.xiaoshuowu.com/robots.txt")
except urllib.error.URLError as e:
    print(e)

使用异常处理模板优先判断是否为http错误，再判断url错误，如果没有错误就输出请求成功
有时reason返回的也可能不是字符串而是对象

In [ ]:
try:
    response = urllib.request.urlopen("http://www.xiaoshuowu.com/robots.txt")
except urllib.error.HTTPError as e:
    print("httperror")
    print(e.reason,e.code,e.headers,sep="\n")
except urllib.error.URLError as e:
    print("urlerror")
    print(e.reason, e.code, e.headers, sep="\n")
else:
    print("request success")    

# urllib分析url

In [2]:
import urllib.parse

使用urlparse类来分析url内容 scheme协议 netloc域名 path访问路径 paeams参数 query查询条件 fragment锚点

`urlparse（urlstring，scheme="http|https"，allow——fragments=True）`

urlstring带解析的url scheme默认的协议（只有在url中没有表明使用）all_fragments忽略锚点（设置成False忽略）

可以使用索引获得result的指定属性 或者result.scheme来获取

In [3]:
result=urllib.parse.urlparse("http://www.baodu.com/index.html;user?id=5#comment")
print(result)

ParseResult(scheme='http', netloc='www.baodu.com', path='/index.html', params='user', query='id=5', fragment='comment')


urlparse是分解url，相对的urlunparse就是并接成url，传入长度为6元素的列表或元组

urlsplit就是urlparse没有paeams将其合并到path中

urlunsplit相当于urlunparse不过传入数据必须长度为5个元素

urljoin（原url，新rul）会将新url的内容拼接到基础url上，基础url只为新url提供scheme协议 netloc域名 path访问路径

但当新url中有这些内容时就使用新url的

In [4]:
data=['http', 'www.baodu.com', 'index.html', 'user', 'id=5', 'comment']
url=urllib.parse.urlunparse(data)
print(url)

http://www.baodu.com/index.html;user?id=5#comment


urlencode将字典构造到url中

parse_qs就是urlencode的反序转换回字典

parse_qsl与parse_qs一样只不过输出元组列表

quote将内容转换为url的编码格式

unquote将url编码

In [5]:
data={"name":"a","age":"18"}
bas_url="http://www.baodu.com?"
new_url=bas_url+urllib.parse.urlencode(data)
print(new_url)

http://www.baodu.com?name=a&age=18


In [6]:
data="你好"
urllib.parse.quote(data)

'%E4%BD%A0%E5%A5%BD'

In [7]:
urllib.parse.unquote(urllib.parse.quote(data))

'你好'

# urllib爬虫协议

user-agent:爬虫名称

Disallow:禁止爬取的路径

Allow:允许的路径

*代表所有

In [9]:
import urllib.robotparser
rp=urllib.robotparser.RobotFileParser()
rp.set_url("https://www.jd.com/robots.txt")
#设置url，也可以在生成实例时设定
rp.read()
# 读取分析robots.txt

也可以省略ser_url与read使用parse（robots.txt的str）

可以通过can_fetch（爬虫名称，需要爬取的url）返回的bool值判断url是否可爬

In [10]:
can=rp.can_fetch("*","https://www.jd.com")
print(can)

True


返回上一次爬取的时间，时间戳

In [11]:
time=rp.mtime()
print(time)

1620314615.321051
